In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import datetime
from bs4 import BeautifulSoup

def scrape_page(driver, url):
    driver.get(url)
    
    # Wait for JavaScript to load (adjust the time as needed)
    driver.implicitly_wait(30)

    # Check for specific content indicating a 403 error
    if "access denied" in driver.page_source.lower():
        print("403 Forbidden: Access Denied")
        return None

    
    # Use BeautifulSoup to parse the page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Find the table containing the applications
    table = soup.find('table', {'id': 'ctl00_Content_cusResultsGrid_repWebGrid_ctl00_grdWebGridTabularView'})
    print(table)
    
    applications = []
    if table:
        rows = table.find_all('tr', class_=['normalRow', 'alternateRow'])
        for row in rows:
            columns = row.find_all('td')
            application = {
                'council_reference': columns[0].text.strip(),
                'date_received': columns[1].text.strip(),
                'description': columns[2].text.strip(),
                'address': columns[3].text.strip(),
                'stage': columns[4].text.strip(),
                'info_url': 'URL Construction Logic Here',  # Placeholder for info URL logic
                'date_scraped': datetime.datetime.now().strftime('%Y-%m-%d')
            }
            applications.append(application)
    
    return applications

def main():
    start_url = "https://eservices.yarracity.vic.gov.au/WebApps/eProperty/P1/eTrack/eTrackApplicationSearch.aspx?Custom=Yes&ApplicationID=P&r=P1.WEBGUEST&f=P1.ETR.SEARCH.ALL"
    
    # Set up Selenium WebDriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode (without opening a browser window)
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
    chrome_options.add_argument("--referrer=https://www.google.com")
    # s = Service('/path/to/chromedriver')  # Replace with the path to your ChromeDriver

    with webdriver.Chrome(options=chrome_options) as driver:
        applications = scrape_page(driver, start_url)
        if applications:
            for app in applications:
                print(app)


main()
